# Trabalho Prático 1
**Grupo 22**

Alexis Correia - A102495 <br>
João Fonseca - A102512 <br>


## Exercício 1
Pretende-se construir um `horário semanal` para o plano de reuniões de projeto de uma “StartUp” de acordo com as seguintes condições:

1. Cada reunião ocupa uma **sala** (enumeradas 1...S) durante um **slot** (tempo e dia).  Assume-se os **dias** enumerados 1..D e, em cada dia, os **tempos** enumerados 1..T.

2.  Cada reunião tem associado um **projeto** (enumerados 1..P) e um conjunto de participantes. Os diferentes **colaboradores** são enumerados 1..C.

3. Cada projeto tem associado um conjunto de colaboradores, dos quais um  é o líder. Cada projeto realiza um dado número de reuniões semanais. São `inputs` do problema o conjunto de colaboradores de cada projeto, o seu líder e o número de reuniões semanais.

4. O líder do projeto participa em todas as reuniões do seu projeto; os restantes colaboradores podem ou não participar consoante a sua disponibilidade, num mínimo (“quorum”) de  **50\%** do total de colaboradores do projeto.  A disponibilidade de cada participante, incluindo o lider,  é um conjunto de “slots” (`inputs` do problema).

### Resolução


In [16]:
#Inicialização
from ortools.linear_solver import pywraplp
import random

solver = pywraplp.Solver.CreateSolver('SCIP')
error = pywraplp.Solver.INFEASIBLE


In [17]:
# Constantes

S,D,T = 5,5,8 #num de salas, dias na semana e tempos(horários) por dia
P,C = 5,20 #num de projetos e de colaboradores totais
cp = 5 # constante auxiliar - num d colaboradores por projeto

In [ ]:
#Gerador aleatório para testes

Slots = [(d, t) for d in range(D) for t in range(T)] # Slots, formados por dia e tempo
aux = set(range(C)) # dicionário com os colaboradores "livres"
HC = [random.sample(Slots, 20) for _ in range(C)] #Horário com a disponibilidade de cada Colaborador

Proj = [] #Lista dos projetos, com pares formados por: (num de reuniões, lista de colaboradores)
for _ in range(P):
  cs = random.sample(aux, cp) #Seleciona colaboradores aleatórios para o projeto
  r = random.randint(1, 5) # Decide o num de reunioes de cada projeto (de 1 a 5)
  l = random.choice(cs) # Escolhe o líder de cada projeto dentre os colaboradores do projeto
  Proj.append((r,l, cs)) # Adiciona a lista dos projetos
  aux = aux - set(cs)
    #retira os colaboradores do dicionário, garantindo que cada colaborador trabalha em apenas um projeto

for c in range(30): #Imprime o horário de todos os colaboradores
  HC[c].sort()
  print(c, HC[c])

for r, l, cs in Proj: #Imprime a lsita Proj
  print(r, l, cs)

In [19]:
# Criação da matriz multi-dimensional
X = {}
for s in range(S):
    for d in range(D):
        for t in range(T):
            for p in range(P):
                for c in range(C):
                    X[s,d,t,p,c] = solver.BoolVar(f"x[{s},{d},{t},{p},{c}]")

Cada projeto tem associado um conjunto de colaboradores, dos quais um  é o líder. Além disso, o líder do projeto participa de todas as reuniões do seu projeto. Ou seja, tomando $L_{p}$ como o líder dum projeto $p$:
$$
\forall p \in P, \sum_{s\in S,d\in D,t\in T}X_{s,d,t,p,L_p} = R
$$

Para além do líder, os outros colaboradores podem ou não participar consoante a sua disponibilidade. No estanto, é preciso um mínimo (“quorum”) de  50\% do total de colaboradores do projeto na reunião.
$$
\forall p \in P,s\in S,d\in D,t\in T. \sum_{c\in C} X_{s,d,t,p,c} \geq C/2
$$

In [20]:
#Restrições:

# 1 - O líder tem de estar em todas as R reuniões do seu projecto
for p in range(P):
  solver.Add(sum(X[s, d, t, p, Proj[p][1]] for s in range(S) for d in range(D) for t in range(T)) == Proj[p][0])

# 2 - Slot (d, h) fora da disponibilidade do colaborador, logo não pode ser usado
for s in range(S):
  for d in range(D):
    for t in range(T):
      for p in range(P):
        for c in range(C):
          if (d, t) not in HC[c]:
            solver.Add(X[s, d, t, p, c] == 0)

# 3 - Colaboradores que não são do projecto não podem estar nele
for s in range(S):
  for d in range(D):
    for t in range(T):
      for p in range(P):
        for c in range(C):
          if c not in Proj[p][2]:
            solver.Add(X[s, d, t, p, c] == 0)

# Limitações:

# 4 - Cada sala tem alocada, no máximo, um projeto
for s in range(S):
  for d in range(D):
    for t in range(T):
      solver.Add(sum(X[s, d, t, p, Proj[p][1]] for p in range(P)) <= 1)

# 5 - Cada colaborador de um projeto só pode estar numa sala
for d in range(D):
  for t in range(T):
    for p in range(P):
      for c in Proj[p][2]:
        solver.Add(sum(X[s, d, t, p, c]  for s in range(S)) <= 1)

# Obrigações:

# 6 - Participação de 50% com o líder incluido
for s in range(S):
  for d in range(D):
    for t in range(T):
      for p in range(P):
        solver.Add(sum(X[s, d, t, p, c] for c in Proj[p][2]) >= 3 * X[s, d, t, p, Proj[p][1]])

Agora, com as restrições adicionadas ao solver, podemos resolver o problema.

In [ ]:
# Fazer o solve - Resolução

status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
  for p in range(P):
    print("Projeto", p+1) 
    presencas = []
    for dia in range(D):
      print("Dia {:<14}", dia+1)
    print()
    for tempo in range(T):
      for dia in range(D):
        print("||", tempo+1, end=" || ")
        for s in range(S):
          if round(X[s, dia, tempo, p, Proj[p][1]].solution_value()) == 1:
            presencas.append([c for c in range(C) if round(X[s, dia, tempo, p, c].solution_value())])
            print(s, end=" ")
          else:
            print("x", end=" ")
      print()
    print(presencas)
else:
  print("impossível")